# Fetching Tweets in Real Time using Twitter API

Tweepy library is used to route all the Live tweets to a TCP socket server.<br> 
This server is hosted in same local machine.

**Import tweepy library**

In [1]:
! pip3 install tweepy --user

     |████████████████████████████████| 77 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 63 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 151 kB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 36.4 MB/s eta 0:00:01


In [2]:
import tweepy

**Import OAuthHandler**<br> Used for Authentication protocol (OAuth) of Twitter API.

In [3]:
from tweepy import OAuthHandler

**Import Stream & StreamListner**<br> Listen to streaming data getting from the socket.

In [5]:
from tweepy import Stream

**Import socket** <br> It is used to create socket connection with Localhost i.e. local server. <br>
**Import Json** <br>Json is used because twitter data comes in json format.

In [6]:
import socket
import json

To connect to Twitter API, we need these 4 Keys. <br>
It can be generated from https://apps.twitter.com/ by creating a new app.

In [7]:
consumer_key = 'cJxdE5SpqN6td3L34QjhqZF6j'
consumer_secret = '4D6nrOWIsAnfB4Eojiaw62K1oHR0WiR27hXu9U0nRMR00kFYXu'
access_token = '969589729527185408-HgwCsWXW5Y5DT5rLv6EaXWkfjY4Nqb0'
access_secret = '41CkS9c4d0CXSovhDipl5pDimgkBMsqmX7NeyAAI0kgQe'

#### Class TweetsListner
    1.listening to the stream via StreamListener. 
    2.constrctor function (init) : initializing the socket. 
    3.tweet_data contains the tweet's json coming from the stream. 
    4.In the tweet's json object 'text' field contains the actual Tweet.
    5.The actual tweet is extracted & sent to the client socket.
    6.on_error is used to catch & throw errors.

In [8]:
class TweetsListener(tweepy.Stream):
    def __init__(self, csocket):
      self.client_socket = csocket
        
    def on_data(self, tweet_json):
        try:
            tweet_data = json.loads( tweet_json )
            print("tweet_text: ", tweet_data['text'].encode('utf-8') )
            print("created_at: ", tweet_data['created_at'])  
            print("name: ", tweet_data['user']['name'])              
            print("location: ", tweet_data['user']['location'])
            print("\n")
            s = tweet_data['text'] + "\n"
            print(s)
            s = s.encode('utf-8')
            self.client_socket.send( s )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True

### Function send_twitter_data():
    1. auth is doing Authentication check using OAuthHandler. It uses 4 keys of Twitter APIs.
    2. consumer_key & consumer_secret are like username & access_token & access_secret are like password.
    3. twitter_stream will get the actual twitter live stream data.
    4. It'll call Stream with auth & call the Class TweetsListner & pass c_socket info to the class.
    5. From the stream of tweets, filter & take only tweets which contains "corona" word.

In [10]:
def send_twitter_data(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    twitter_stream_data = Stream(auth, TweetsListener(c_socket))
    twitter_stream_data.filter(track=['corona'])

### Socket Creation:
    1. Socket is created by using socket function.
    2. Host is localhost, which is machine IP, where this Jupyter notebook is running.
    3. Port is 7773 (It can be anything, unique for this service)
    4. Now bind host & port address to the socket.

In [11]:
# Create a socket object
s = socket.socket()

# Get local machine name
host = "127.0.0.1"

# Reserve a port for your service.
port = 7777

# Bind to the port
s.bind((host, port))

# print
print(f"Listening on port: {str(port)}")

Listening on port: 7777


socket will wait & listen for few seconds. <br>
After that connection is made.

In [12]:
# Now wait for client connection.
s.listen(5)

In [ ]:
# Establish connection with client.
c, addr = s.accept()
print(f"Recieved response from: {str(addr)}")

### Send live Tweets in real time to the Socket
Function send_twitter_data will send all the Tweets containing word 'corona' to socket 'c' in real time.<br>
This will be consumed in the next code of Spark Streaming to do Trend Analysis.